# Append, Concat, & Merge 

Estas tres funciones de pandas son de suma importancia al trabajar con varias bases de datos a la vez, son las herramientas de las que haremos uso al tener que unir `DataFrames`, y haremos un repaso en como usar estas funciones.

## Append

`Append` a mi pareces es la más intuitiva y simple de las tres, esta la usamos con el fin de agregar columnas a nuestro `DataFrame`

In [1]:
import pandas as pd
import numpy as np

Crearemos dos dataframes que tengan las mismas columnas

In [2]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df

,A,B
0,1,2
1,3,4


In [3]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
df.append(df2)

,A,B
0,1,2
1,3,4
0,5,6
1,7,8


Podemos ver que las filas fueron agregadas al final, notemos que el indice no se cambió, por lo que tenemos indices repetidos

In [4]:
df.append(df2).loc[0]

,A,B
0,1,2
0,5,6


Pandas puede manejar esto, y al invocar ese indice nos entrega ambos valores, pero en general queremos que el indice sea único para cada valor, para esto podemos usar `ignore_index=True`, que hace lo que dice en el nombre

In [5]:
df.append(df2, ignore_index=True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


Tambien es bueno saber que, si usamos `append` para `DataFrames` con columnas distintas, los valores "faltantes" seran rellenados con `NaN`, en ambas direcciones

In [6]:
df3 = pd.DataFrame([[5, 6, 'a'], [7, 8, 'b']], columns=list('ABC'))
df3

,A,B,C
0,5,6,a
1,7,8,b


In [7]:
df.append(df3, ignore_index=True) #agregando con mas columnas

,A,B,C
0,1,2,NaN
1,3,4,NaN
2,5,6,a
3,7,8,b


In [8]:
df3.append(df, ignore_index=True) #agregando con menos columnas

,A,B,C
0,5,6,a
1,7,8,b
2,1,2,NaN
3,3,4,NaN


## Concat

`Concat` es similar a `Append`, pero más poderoso. En buenas a primeras parece que hacen lo mismo, agregar una `DataFrame` al final de otro

In [9]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
pd.concat([s1, s2])

0    a
1    b
0    c
1    d
dtype: object

Y de la misma forma tenemos que usar `ignore_index=True` si es que los datos compartian indices

In [10]:
pd.concat([s1, s2], ignore_index=True)

0    a
1    b
2    c
3    d
dtype: object

Pero la primera gran diferencia es que esta funcion tiene un entendimiento de que los `DataFrames` que estamos juntando son distintos, motivo por el cual podemos entregarle el argunmento `keys`, que nos permite usar indices multiples para referirnos a cada `Dataframe` que usamos

In [11]:
pd.concat([s1, s2], keys=['s1', 's2'])

s1  0    a
    1    b
s2  0    c
    1    d
dtype: object

In [12]:
df1 = pd.DataFrame([['a', 1], ['b', 2]]
                  , columns=['letter', 'number']
                 )
df1

,letter,number
0,a,1
1,b,2


In [13]:
df2 = pd.DataFrame([['c', 3], ['d', 4]]
                   , columns=['letter', 'number']
                  )
df2

,letter,number
0,c,3
1,d,4


In [14]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']] 
                   , columns=['letter', 'number', 'animal']
                  )
df3

,letter,number,animal
0,c,3,cat
1,d,4,dog


La otra gran diferencia, es que podemos unir varias `DataFrames` a la vez, y notemos que tambien rellenan los `NaN` 

In [15]:
pd.concat([df1, df2 ,df3], sort=False)

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
0,c,3,NaN
1,d,4,NaN
0,c,3,cat
1,d,4,dog


## Merge

`Merge` es, a mi parecer, la más compleja de las dos. No porque es dificl de entender, sino por el millar de formas que hay de hacerlo. Hoy explicaré las 4 principales: `inner`, `left`, `rigth`, y `outer`, pero hay varias más. Para esto la documentacion de *Pandas* no es lo mejor, así que les dejo esta respuesta de [*StackOverflow*](https://stackoverflow.com/questions/53645882/pandas-merging-101) (que es de donde saqué las imagenes) por si quieres ver el resto de las posibilidades.

Crearemos dos `Dataframes` distintos y veremos cuales son las formas de hacer un `Merge`

### Inner

In [16]:
np.random.seed(0)
left = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})    
right = pd.DataFrame({'key': ['B', 'D', 'E', 'F'], 'value': np.random.randn(4)})

<img src="images/inner.png" width="400">

In [17]:
left.merge(right, on='key') 
# inner es el valor por defecto pero podemos ser más especificos 
# left.merge(right, on='key', how='inner')

,key,value_x,value_y
0,B,0.400157,1.867558
1,D,2.240893,-0.977278


### Left

<img src="images/left.png" width="400">

In [18]:
left.merge(right, on='key', how='left')

,key,value_x,value_y
0,A,1.764052,NaN
1,B,0.400157,1.867558
2,C,0.978738,NaN
3,D,2.240893,-0.977278


### Rigth

<img src="images/rigth.png" width="400">

In [19]:
left.merge(right, on='key', how='right')

,key,value_x,value_y
0,B,0.400157,1.867558
1,D,2.240893,-0.977278
2,E,NaN,0.950088
3,F,NaN,-0.151357


### Outer

<img src="images/outer.png" width="400">

In [20]:
left.merge(right, on='key', how='outer')

,key,value_x,value_y
0,A,1.764052,NaN
1,B,0.400157,1.867558
2,C,0.978738,NaN
3,D,2.240893,-0.977278
4,E,NaN,0.950088
5,F,NaN,-0.151357


# Actividad

Como ya es común, usaremos una base de datos que podemos en contrar en Kaggle, esta ves será ["Restaurant Data with Consumer Ratings"](https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings?select=rating_final.csv). Citando a Kaggle:

*This dataset was used for a study where the task was to generate a top-n list of restaurants according to the consumer preferences and finding the significant features. Two approaches were tested: a collaborative filter technique and a contextual approach: (i) The collaborative filter technique used only one file i.e., rating_final.csv that comprises the user, item and rating attributes. (ii) The contextual approach generated the recommendations using the remaining eight data files.*

Los `.csv` que usaremos están en la carpeta `data`

Para las actividades que veremos, nos interesa tener el *rating* promedio tanto de los restaurantes como de los usuarios, motivo por el cual les daré estos dos dataframes ya listos. Si ya terminaste con la actividad, y te sientes aventurero, puedes tratar de fabricarlos tú.

In [ ]:
df = pd.read_csv('./data/rating_final.csv')
df.head()

In [ ]:
user_mean = (df.drop('placeID',axis=1) #saco la columna de restaurantes
               .groupby('userID') #junto a todos los usurarion
               .apply(lambda x : x.mean()) #le saco el promedio a cada atributo
            )
user_mean.head()

In [ ]:
res_mean = df.drop('userID',axis=1).groupby('placeID').apply(lambda x : x.mean()).drop('placeID',axis=1) #pos basicamente lo mismo de arriba
res_mean.head()

Esto dejará de ser magia oscura en algún momento, lo prometo! pero de todas formas les dejo una explicación a *grosso modo* de que es lo que se está ahciendo. 
Ultima cosa antes de ir al ejercicio, un metodo muy util cuando trabajamos con dataframes de datos numericos es el metodo `.describe()`, lo podemos ver a continuacion:

In [ ]:
res_mean.describe()

Su funcion es bastante autoexplicativa, nos entrega un resumen con estadisticos que podemos usar.

Ahora tenemos lo que queremos `user_mean` y `res_mean`, entonces ¿Qué podemos hacer con estos datos?. Veamos primero el dataset con mas columnas. 

In [ ]:
places = pd.read_csv('./data/geoplaces2.csv')
places.columns

Como suele ocurrir, muchas de estás comlumnas no vamos a usar así que nos desaremos de ellas 

In [ ]:
no_use = ['latitude', 'longitude', 'the_geom_meter','address','city', 'state', 'fax', 'zip','url']
places = places.drop(no_use, axis = 1)
places.head()

Mucho más trabajable, para ver el detalle de las columnas, en la carpeta data hay un `README.txt`, este explica que es cada archivo y cuales son sus columnas. Ahora, nos enfocaremos en 3 `alcohol`, `price`, y `dress_code`. Veremos si es que hay grandes diferencias entre cada una de estas opciones y su  valoracion promedio. 

Pero antes, tenemos un ID para cada restaurante, la columna `placeID` , ya que tenemos una sola fila por cada uno, podemos usar esta como indice:

In [ ]:
places = places.set_index(#fill)
places.head()

Es claro que para hacer esto de una forma rápida, sería conveniente tener una columna que sea cada *rating*, y para esto usaremos `merge`.   Una cosa que no mencioné durante la explicación de `merge` es que podemos hacer sobre los incides, para esto tenemos que especificar que indice queremos usar, y esto lo hacemos con los parametros `left_index = True` y `right_index = True`. Ahora, ya que tenemos `places` y `res_mean` que tienen el mismo indice, hagamos un merge entre las dos:

In [ ]:
places = places.merge(#fill
                      , left_index = #fill
                      , right_index = #fill
                      , how = #fill
                     )
places.head()

Primero revisaremos si el expendio de alcohol afecta los *ratings* que recibe cada restaurante, esta columna puede tomar 3 valores `No_Alcohol_Served`, `Wine-Beer`, y `Full_Bar`, esto lo podmeos ver de forma rapida usando el metodo `.unique`

In [ ]:
places['alcohol'].unique()

Ahora, selecionando solo 

In [ ]:
places[places['#fill']=='#fill'][['#fill', '#fill', '#fill']].#fill()

In [ ]:
places[places['#fill']=='#fill'][['#fill', '#fill', '#fill']].#fill()

In [ ]:
places[places['#fill']=='#fill'][['#fill', '#fill', '#fill']].#fill()

Es claro que esto no es un test de hipotesis (pero, tenemos de todo como para hacer uno), pero de buenas a primeras, ¿Crees que hay una direrencia significativa entre las valoraciones?

**Respuesta**: 

Ahora, prueba con otro atributo, ¿alguno que te llame la atención? Haz un proceso similar para otra variable que te llame la atención

In [ ]:
#fill

**Respuesta**

Ahora, veamos como se relaciona todo con si los restaruantes tienen estacionamiento:

In [ ]:
park = pd.read_csv('./data/chefmozparking.csv')
park.head()

Veamos que valores pueden tomar estos datos

In [ ]:
park['parking_lot'].unique()

Ahora, de la misma forma haremos un merge, pero esta vez no pondremos la fila `placeID` como indice, para mostrar que podemos hacer el merge entre indices y colunas, para esto usamos ` left_on = 'nombre_col'` (o si la columnas que queremos usar esta en lado derecho `rigth_on = ...`). Por tanto, en este merge le tenemos que decir al merge que lo haga en la columna izquierda `placeID` y en la derecha que use el indice

In [ ]:
park = park.merge(#fill
                  ,  left_on = #fill
                  ,  right_index = #fill
                 )
park.head()

Acá, ¿hay algo que te produsca curiosidad?, hay demasiados valores unicos como para revisarlos uno por uno, asi que usa dos o tres que te llamen la atencion y revisa como re comparan entre ellos

In [ ]:
#fill

**Respuesta**

Ahora lo dejo en tus manos, aun no hemos tocado a los usuarios, pero eso te lo dejo a ti. Tenmos la base de datos `userprofile.csv`, que es muy interesante:

In [ ]:
profile = pd.read_csv('./data/userprofile.csv')
profile.columns

Podemos ver que hay mucha informacion sobre los usuarios, te invito a tomar dos variables y explorar como estas se relacionan con la puntuacion promedio de cada persona, dejo a tu elección si quieres hacer un `set_index`, pero tienes que hacer al menos un `merge`, o dos si te sientes aventurero. Sean creativos! jueguen con los datos y vean a que pueden llegar. 

In [ ]:
#fill

**Conclusion**

Sé que lo que hicimos fue superficial, y que los datos dan para mucho más, así que en la proxima ayudantía, ya armados con más herramientas, espero que podamos volver a estos datos y hacer cositas más interesantes!